In [ ]:
#| default_exp markdown.obsidian.personal.machine_learning.notation_summarization

# markdown.obsidian.personal.machine_learning.notation_summarization
> Functions for summarizing notations

In `markdown.obsidian.personal.machine_learning.notation_identification`, we defined functions to gather ML data to identify notations and functions to use an ML model to predict where notations are defined. In `how_to.train_ml_model`, we showed how to train this ML model from the data. `markdown.obsidian.personal.notation` also has some functions to create and interact with notation notes.

It would be further convenient to train an ML model to summarize details about notations. This module defines functions to gather data to train such a model.

In [ ]:
#| export
import os
from os import PathLike
from typing import Union

import pandas as pd

from trouver.helper import current_time_formatted_to_minutes
from trouver.markdown.markdown.file import MarkdownFile
from trouver.markdown.obsidian.personal.machine_learning.database_update import append_to_database
from trouver.markdown.obsidian.personal.note_processing import process_standard_information_note
from trouver.markdown.obsidian.personal.notation import parse_notation_note
from trouver.markdown.obsidian.personal.note_type import note_is_of_type
from trouver.markdown.obsidian.vault import VaultNote

In [ ]:
from unittest import mock
import shutil
import tempfile

from fastcore.test import *
# from torch import Tensor

from trouver.helper import _test_directory

# Gather ML data from notation notes

In [ ]:
#| export
def notation_summarization_data_from_note(
        notation_note: VaultNote,
        vault: PathLike
        ) -> Union[dict[str, str], None]: # The keys to the dict are "Notation note name", "Notation", "Latex in oiriginal", "Summary" The 
    """Obtain notation summzarization data from the notation note
    if the notation is determined to have been summarized and if
    the main note of the notation note exists.

    The notion of whether the notation "has been summarized" is not
    exactly implemented, but should be sufficient for gathering
    data.

    TODO: there are some notation notes drawing information from
    multiple notes; separate text in notation notes corresponding
    to each note.

    **Returns**

    - Union[dict[str, str], None]

        - If the notation note is determined to have been summarized
          (i.e. contains text beyond `<notation> denotes ` and does not
          have the `#_meta/TODO` tag) then the output is a `dict` whose
          key-value pairs are
            - `"Notation note name"` - The name of the notation note
            - `"Notation"` - The notation of the note
            - `"Latex in original"' - The entry of the `latex_in_original`
              field of the note if available, cf. `make_a_notation_note`
            - `"Summary"` - The summary of the notation.
            - `"Main note name"` - The name of the main note of the
              notation note
            - `"Processed main note contents"` - The processed contents of the
              main note
        - Otherwise, the output is `None.
    """
    metadata, notation_str, main_of_notation, main_mf, _ = parse_notation_note(notation_note, vault)
    if not _notation_has_been_summarized(main_mf):
        return None

    if metadata is not None and 'latex_in_original' in metadata:
        latex_in_original = metadata['latex_in_original']
        if isinstance(latex_in_original, list):
            latex_in_original = latex_in_original[0]
    else:
        latex_in_original = None

    process_standard_information_note(main_mf, vault)
    processed_summary = str(main_mf)

    if main_of_notation is None:
      return None
    main_note = VaultNote(vault, name=main_of_notation)
    if not main_note.exists():
        return None
    main_note_mf = MarkdownFile.from_vault_note(main_note)
    process_standard_information_note(main_note_mf, vault)

    return {"Notation note name": notation_note.name,
            "Notation": notation_str,
            "Latex in original": latex_in_original,
            "Summary": processed_summary,
            "Main note name": main_of_notation,
            "Processed main note contents": str(main_note_mf)}


def _notation_has_been_summarized(
        main_mf: VaultNote
        ) -> bool:
    text = str(main_mf).strip()
    return len(text) > 0 and '#_meta/TODO' not in text


In [ ]:
vault = _test_directory() / 'test_vault_7'
notation_note = VaultNote(vault, name='some_reference_name_notation_Spec_A') 
sample_output = notation_summarization_data_from_note(notation_note, vault)

del sample_output['Processed main note contents']

test_eq(sample_output,
    {'Notation note name': 'some_reference_name_notation_Spec_A',
     'Notation': '$\\operatorname{Spec} A$',
     'Latex in original': '\\operatorname{Spec} A',
     'Summary': 'the spectrum of the ring $A$.',
     'Main note name': 'spectrum_of_a_ring',
     # 'Processed main note contents': 'Let $A$ be a ring.\n\nThe Spectrum $\\operatorname{Spec} A$ is the set of prime ideals of $A$. It is equipped with a topology, called the Zariski topology in which the followings sets, called the distinguished open subsets of $\\operatorname{Spec} A$, give a base for the topology:\n\n$$D(f) = \\{\\mathfrak{p} \\in \\operatorname{Spec} A: f \\not\\in \\mathfrak{p} \\}.$$\n'}
    })

If the "main" note of the notation note cannot be identified, then `notation_summarization_data_from_note` returns `None`:

In [ ]:
# vault = _test_directory() / 'test_vault_7'
# notation_note = VaultNote(vault, name='some_reference_name_notation_B_R') 
# test_eq(notation_summarization_data_from_note(notation_note, vault), None)

# TODO: example of a notation note which has a main not, but the main note does not exist
# TODO: example of a notation note without a main note at all.

In [ ]:
# TODO: example with embedded links

In [ ]:
# TODO: example of a notation note without any metadata

In [ ]:
#| export
def gather_notation_note_summaries(
        vault: PathLike,
        notes: list[VaultNote]
        ) -> pd.DataFrame: # Has columns `Time added`, `Time modified`, `Notation note name`, `Notation`, `Latex in original`, 'Summary', 'Main note name', and 'Processed main note contents'. 
    """
    Return a `pandas.DataFrame` encapsulating the data of notation note
    summaries.

    cf. 
    """
    summary_data = [
        notation_summarization_data_from_note(note, vault) for note in notes]
    summary_data = [row for row in summary_data if row is not None]
    current_time = current_time_formatted_to_minutes()
    for row in summary_data:
        row['Time added'] = current_time
        row['Time modified'] = current_time
    return pd.DataFrame(summary_data)
    

In [ ]:
# TODO: example

In [ ]:
# TODO: example verifying that outputs of `None` from `notation_summarization_data_from_note`
# are weeded out.

# Use ML model to fill in notation notes

In [ ]:
#| export
def append_to_notation_note_summarization_database(
        vault: PathLike, # The vault freom which the data is drawn
        file: PathLike, # The path to a CSV file
        notes: list[VaultNote], # the notation notes to consider adding to the database. 
        backup: bool = True # If `True`, makes a copy of `file` in the same directory and with the same name, except with an added extension of `.bak`.
        ) -> None:
    """
    Either create a `csv` file containing data for information note type
    labels or append to an existing `csv` file.

    The columns of the database file are as follows:

    - `Time added` - The time when the row was added.
    - `Time modified` - The time when the labels of the row 
    - `Notation note name` - The name of the note from which the data for the row
      was derived.
    - 'Notation' - The notation which is being summarized
    - 'Latex in original' - The entry of the `latex_in_original` field of the
      note if available, cf. `make_a_notation_note`
    - `"Summary"` - The summary of the notation.
    - `"Main note name"` - The name of the main note of the
      notation note
    - `"Processed main note contents"` - The processed contents of the
      main note

    All timestamps are in UTC time and specify time to minutes
    (i.e. no seconds/microseconds).
    
    TODO: implement updating rows and rewrite the next paragraph to
    accurately reflect the implementation. I would like the 'Notation', 'Latex in original',
    'Summary', 'processed main note contents' to be the "pivot_cols"

    If a "new" note has the same processed content as a pre-existing
    note and anything is different about the "new" note, then update
    the row of the existing note. In particular, the following are updated:
    - Time modified (set to current time)
    - Notation (overwritten)
    - Latex in original (overwritten)
    - Summary (overwritten)
    - Main note name (overwritten)
    - Processed main note contents (overwritten)
    
    This method assumes that all the processed content in the
    CSV file are all distinct if the CSV file exists.
    """
    if not notes:
        return
        file = Path(file)
    df = pd.read_csv(file) if os.path.exists(file) else None
    # start_ID_from = max_ID(df) + 1 if not df is None else 1
    new_df = gather_notation_note_summaries(vault, notes)
    cols = [
        'Time added', 'Time modified', 'Notation note name',
        'Notation', 'Latex in original', 'Summary', 'Main note name',
        'Processed main note contents']
    cols_to_update = [
      'Time modified', 'Notation note name', 'Notation', 'Latex in original', 'Summary', 'Main note name']
    append_to_database(
        file, new_df, cols, 'Processed main note contents', cols_to_update, backup)

In [ ]:
# TODO basic example

In [ ]:
# TODO: example of appending instead of making an entirely new CSV file

> 